# Preprocessing data

## Eco2mix

In [71]:
REGIONS = [
    "Auvergne-Rhône-Alpes",
    "Bourgogne-Franche-Comté",
    "Bretagne",
    "Centre-Val-de-Loire",
    "Grand-Est",
    "Hauts-de-France",
    "Ile-de-France",
    "Normandie",
    "Nouvelle-Aquitaine",
    "Occitanie",
    "PACA",
    "Pays-de-la-Loire"
]

FILETYPES = [
    "En-cours-Consolide",
    "En-cours-TR",
    "Annuel-Definitif_2013",
    "Annuel-Definitif_2014",
    "Annuel-Definitif_2015",
    "Annuel-Definitif_2016",
    "Annuel-Definitif_2017",
    "Annuel-Definitif_2018",
    "Annuel-Definitif_2019",
    "Annuel-Definitif_2020",
]


In [2]:
import warnings
import pandas as pd
import numpy as np
import os

warnings.filterwarnings("ignore")

In [6]:
#loop over regions and filetypes to create different dataframes
concat_df = pd.DataFrame()
concat_df2 = pd.DataFrame()
concat_df3 = pd.DataFrame()

for region in REGIONS:
    for filetype in FILETYPES:
        filename = f"./data/eco2mix/Auvergne-Rhône-Alpes/{filetype}.txt"
        df = pd.read_csv(filename, sep="\t", encoding="latin-1")
        #delete last row with comment of rte
        df = df[:-1]
        if filetype != "Annuel-Definitif_2019":
            #delete all columns except Date, Heure and Consommation
            df = df[["Date", "Heures", "Consommation"]]
        else:
            #delete all columns except Date, Heure and Consommation
            df = df[["Périmètre", "Nature", "Date"]]
        #rename columns
        df.columns = ["Date", "Heures", f"Consommation_{region}"]
        #put date and hour in index
        df = df.set_index(["Date", "Heures"])
        #delete all rows with NaN values
        df = df.dropna()

        concat_df = pd.concat([concat_df, df], axis=0)
print(concat_df)
    #concat_df2 = pd.concat([concat_df2, concat_df[f"Consommation_{region}"]], axis=1)
    #concat_df2 = concat_df2.reset_index()
#print(concat_df2)


                  Consommation_Auvergne-Rhône-Alpes  \
Date       Heures                                     
2021-01-01 00:00                             8576.0   
           00:30                             8514.0   
           01:00                             8273.0   
           01:30                             8253.0   
           02:00                             8234.0   
...                                             ...   
2020-12-31 21:30                                NaN   
           22:00                                NaN   
           22:30                                NaN   
           23:00                                NaN   
           23:30                                NaN   

                  Consommation_Bourgogne-Franche-Comté Consommation_Bretagne  \
Date       Heures                                                              
2021-01-01 00:00                                   NaN                   NaN   
           00:30                            

In [165]:
pd.set_option('display.max_columns', None)
print(concat_df.sort_index())

                  Consommation_Auvergne-Rhône-Alpes  \
Date       Heures                                     
2013-01-01 00:00                                 ND   
           00:00                                NaN   
           00:00                                NaN   
           00:00                                NaN   
           00:00                                NaN   
...                                             ...   
2023-12-20 08:30                                NaN   
           08:30                                NaN   
           08:30                                NaN   
           08:30                                NaN   
           08:30                                NaN   

                  Consommation_Bourgogne-Franche-Comté Consommation_Bretagne  \
Date       Heures                                                              
2013-01-01 00:00                                   NaN                   NaN   
           00:00                            

In [83]:
#loop over regions and filetypes to create different dataframes
conso_national = pd.DataFrame()

for filetype in FILETYPES:
    filename = f"./data/eco2mix/France/{filetype}.txt"
    df = pd.read_csv(filename, sep="\t", encoding="latin-1", on_bad_lines='skip')
    df = df[:-1]
    if filetype == "En-cours-Consolide":
        #delete all columns except Date, Heure and Consommation
        df = df[["Date", "Heures", "Consommation"]]
    else:
        #delete all columns except Date, Heure and Consommation
        df = df[["Nature", "Date", "Heures"]]
    #rename columns
    df.columns = ["Date", "Heures", "Consommation"]
    #delete rows heures with a 5 in minutes
    df = df[~df["Heures"].str.contains(":.5")]
    #put date and hour in index
    df = df.set_index(["Date", "Heures"])
    #delete all rows with NaN values
    df = df.dropna()
    conso_national = pd.concat([conso_national, df], axis=0)
print(conso_national.sort_index())

                   Consommation
Date       Heures              
2013-01-01 00:00        61194.0
           00:30        59674.0
           01:00        57877.0
           01:30        57755.0
           02:00        57243.0
...                         ...
2023-12-20 12:00        66012.0
           12:30        64891.0
           13:00        63859.0
           13:30        63326.0
           14:00        63009.0

[187853 rows x 1 columns]


In [80]:
#change index to columns date and heures and keep only rows heures without a 5 in minutes
conso_national = conso_national.reset_index()
conso_national = conso_national[~conso_national["Heures"].str.contains("5")]
conso_national = conso_national.set_index(["Date", "Heures"])
print(conso_national)


                   Consommation
Date       Heures              
2021-01-01 00:00        67010.0
           00:30        67071.0
           01:00        65052.0
           01:30        64918.0
           02:00        64376.0
...                         ...
2020-12-31 21:30        64423.0
           22:00        63505.0
           22:30        64331.0
           23:00        66853.0
           23:30        66750.0

[172199 rows x 1 columns]


In [91]:
FILETYPES = [
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
    "2023",
]

FILES = [
    "01",
    "02",
    "03",
    "04",
    "05",
    "06",
    "07",
    "08",
    "09",
    "10",
    "11",
    "12",
]

In [97]:
meteo = pd.DataFrame()

for filetype in FILETYPES:
    for file in FILES:
        filename = f"./data/meteo/{filetype}/{file}.csv"
        df = pd.read_csv(filename, sep=";")
        df = df[["numer_sta", "date", "t"]]
        #rename columns
        df.columns = ["id_station", "Date", "Temperature"]
        #delete rows by some id_station
        df = df[df.id_station != 7761]
        df = df[df.id_station != 7790]
        df = df[df.id_station != 61968]
        df = df[df.id_station != 61970]
        df = df[df.id_station != 61972]
        df = df[df.id_station != 61976]
        df = df[df.id_station != 61980]
        df = df[df.id_station != 61996]
        df = df[df.id_station != 61997]
        df = df[df.id_station != 61998]
        df = df[df.id_station != 67005]
        df = df[df.id_station != 71805]
        df = df[df.id_station != 78890]
        df = df[df.id_station != 78894]
        df = df[df.id_station != 78897]
        df = df[df.id_station != 78922]
        df = df[df.id_station != 78925]
        df = df[df.id_station != 81401]
        df = df[df.id_station != 81405]
        df = df[df.id_station != 81408]
        df = df[df.id_station != 81415]
        df = df[df.id_station != 89642]
    meteo = pd.concat([meteo, df], axis=0)
print(meteo)


      id_station            Date Temperature
0           7005  20131201000000  278.550000
1           7015  20131201000000  278.450000
2           7020  20131201000000  282.350000
3           7027  20131201000000  277.650000
4           7037  20131201000000  277.750000
...          ...             ...         ...
9138        7643  20231219180000  282.550000
9139        7650  20231219180000  279.850000
9140        7661  20231219180000          mq
9141        7690  20231219180000  282.550000
9142        7747  20231219180000  281.850000

[104485 rows x 3 columns]


In [98]:
#reformat date column in meteo
meteo["Date"] = pd.to_datetime(meteo["Date"], format="%Y%m%d%H%M%S")
#replace mq by NaN
meteo["Temperature"] = meteo["Temperature"].replace("mq", np.nan)
#convert temperature from string to float
meteo["Temperature"] = meteo["Temperature"].astype(float)
#replace nan by mean of temperature before and after
meteo["Temperature"] = meteo["Temperature"].fillna((meteo["Temperature"].shift() + meteo["Temperature"].shift(-1))/2)
#convert temperature from kelvin to celsius
meteo["Temperature"] = meteo["Temperature"] - 273.15

print(meteo)

      id_station                Date  Temperature
0           7005 2013-12-01 00:00:00         5.40
1           7015 2013-12-01 00:00:00         5.30
2           7020 2013-12-01 00:00:00         9.20
3           7027 2013-12-01 00:00:00         4.50
4           7037 2013-12-01 00:00:00         4.60
...          ...                 ...          ...
9138        7643 2023-12-19 18:00:00         9.40
9139        7650 2023-12-19 18:00:00         6.70
9140        7661 2023-12-19 18:00:00         8.05
9141        7690 2023-12-19 18:00:00         9.40
9142        7747 2023-12-19 18:00:00         8.70

[104485 rows x 3 columns]


In [16]:
filename = f"./data/stations_regions.csv"
station_meteo = pd.read_csv(filename, sep=";")
station_meteo = station_meteo[["ID","Region"]]
#delete rows with region = Hors France
station_meteo = station_meteo[station_meteo.Region != "Hors France"]
print(station_meteo)
print(station_meteo.shape)

      ID                   Region
0   7005          Hauts-de-France
1   7015          Hauts-de-France
2   7020                Normandie
3   7027                Normandie
4   7037                Normandie
5   7072                Grand-Est
6   7110                 Bretagne
7   7117                 Bretagne
8   7130                 Bretagne
9   7139         Pays-de-la-Loire
10  7149            Ile-de-France
11  7168                Grand-Est
12  7181                Grand-Est
13  7190                Grand-Est
14  7207                 Bretagne
15  7222         Pays-de-la-Loire
16  7240      Centre-Val-de-Loire
17  7255      Centre-Val-de-Loire
18  7280  Bourgogne-Franche-Comté
19  7299                Grand-Est
20  7314       Nouvelle-Aquitaine
21  7335       Nouvelle-Aquitaine
22  7434       Nouvelle-Aquitaine
23  7460     Auvergne-Rhône-Alpes
24  7471     Auvergne-Rhône-Alpes
25  7481     Auvergne-Rhône-Alpes
26  7510       Nouvelle-Aquitaine
27  7535                Occitanie
28  7558      

In [99]:
#create dataframe with mean temperature per region with use of station_meteo
mean_temp = pd.DataFrame()
for region in REGIONS:
    df = meteo[meteo["id_station"].isin(station_meteo[station_meteo["Region"] == region]["ID"])]
    df = df.groupby(["Date"]).mean()
    df = df.reset_index()
    df = df.set_index(["Date"])
    df = df.rename(columns={"Temperature": f"Temperature_{region}"})
    mean_temp = pd.concat([mean_temp, df], axis=1)
#delete id_station columns
mean_temp = mean_temp.drop(columns="id_station")
mean_temp = mean_temp.resample("30min").interpolate()
print(mean_temp)

                     Temperature_Auvergne-Rhône-Alpes  \
Date                                                    
2013-12-01 00:00:00                          0.950000   
2013-12-01 00:30:00                          0.941667   
2013-12-01 01:00:00                          0.933333   
2013-12-01 01:30:00                          0.925000   
2013-12-01 02:00:00                          0.916667   
...                                               ...   
2023-12-19 16:00:00                          4.608333   
2023-12-19 16:30:00                          4.125000   
2023-12-19 17:00:00                          3.641667   
2023-12-19 17:30:00                          3.158333   
2023-12-19 18:00:00                          2.675000   

                     Temperature_Bourgogne-Franche-Comté  \
Date                                                       
2013-12-01 00:00:00                             1.400000   
2013-12-01 00:30:00                             1.383333   
2013-12-01 01:00:0

In [100]:
#create new dataframe with mean of temperature per date and delete id_station column
national_meteo = meteo.groupby("Date").mean()
national_meteo = national_meteo.drop(columns="id_station")
#create new rows for national_meteo with interpolated temperature values
national_meteo = national_meteo.resample("30min").interpolate()
print(national_meteo.sort_index())

                     Temperature
Date                            
2013-12-01 00:00:00     3.862500
2013-12-01 00:30:00     3.716250
2013-12-01 01:00:00     3.570000
2013-12-01 01:30:00     3.423750
2013-12-01 02:00:00     3.277500
...                          ...
2023-12-19 16:00:00     7.677083
2023-12-19 16:30:00     7.471250
2023-12-19 17:00:00     7.265417
2023-12-19 17:30:00     7.059583
2023-12-19 18:00:00     6.853750

[176197 rows x 1 columns]


In [24]:
DATES = [
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
    "2023",
]

In [66]:
nb_habitants = pd.DataFrame()
for date in DATES :
    filename = f"./data/population.xls"
    df = pd.read_excel(filename, sheet_name=f"{date}", skiprows=4)
    df = df[["Unnamed: 0", "Total"]]
    #rename columns
    df.columns = ["Annee", f"{date}"]
    df = df.drop(df.index[14:])
    df = df[df.Annee != "Corse"]
    df[f"{date}"] = df[f"{date}"].astype(int)
    df = df.set_index(["Annee"])
    df = df.transpose()
    nb_habitants = pd.concat([nb_habitants, df], axis=0)
print(nb_habitants.info())
print(nb_habitants)

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 2013 to 2023
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   Auvergne-Rhône-Alpes        11 non-null     int32
 1   Bourgogne-Franche-Comté     11 non-null     int32
 2   Bretagne                    11 non-null     int32
 3   Centre-Val-de-Loire         11 non-null     int32
 4   Grand Est                   11 non-null     int32
 5   Hauts-de-France             11 non-null     int32
 6   Île-de-France               11 non-null     int32
 7   Normandie                   11 non-null     int32
 8   Nouvelle-Aquitaine          11 non-null     int32
 9   Occitanie                   11 non-null     int32
 10  Pays de la Loire            11 non-null     int32
 11  Provence-Alpes-Côte d'Azur  11 non-null     int32
 12  France métropolitaine       11 non-null     int32
dtypes: int32(13)
memory usage: 660.0+ bytes
None
Annee  Auvergne-Rhône-

In [88]:
nb_habitants.columns = ["Auvergne-Rhône-Alpes", "Bourgogne-Franche-Comté", "Bretagne", "Centre-Val-de-Loire", "Grand-Est", "Hauts-de-France", "Ile-de-France", "Normandie", "Nouvelle-Aquitaine", "Occitanie", "PACA", "Pays-de-la-Loire","France"]
#change index in column and keep just year in index
nb_habitants = nb_habitants.reset_index()
#change format of index to datetime
nb_habitants["index"] = pd.to_datetime(nb_habitants["index"], format="%Y")
#nb_habitants["index"] = nb_habitants["index"].dt.year
nb_habitants = nb_habitants.set_index("index")
nb_habitants = nb_habitants.resample("30min").interpolate()
nb_habitants

,Auvergne-Rhône-Alpes,Bourgogne-Franche-Comté,Bretagne,Centre-Val-de-Loire,Grand-Est,Hauts-de-France,Ile-de-France,Normandie,Nouvelle-Aquitaine,Occitanie,PACA,Pays-de-la-Loire,France
index,,,,,,,,,,,,,
2013-01-01 00:00:00,7.757595e+06,2.819783e+06,3.258707e+06,2.570548e+06,5.552388e+06,5.987883e+06,1.195981e+07,3.328364e+06,5.844177e+06,5.683878e+06,3.660852e+06,4.953675e+06,6.369786e+07
2013-01-01 00:30:00,7.757599e+06,2.819783e+06,3.258708e+06,2.570548e+06,5.552388e+06,5.987884e+06,1.195981e+07,3.328364e+06,5.844179e+06,5.683881e+06,3.660854e+06,4.953677e+06,6.369788e+07
2013-01-01 01:00:00,7.757602e+06,2.819783e+06,3.258709e+06,2.570549e+06,5.552388e+06,5.987885e+06,1.195981e+07,3.328365e+06,5.844181e+06,5.683883e+06,3.660855e+06,4.953678e+06,6.369790e+07
2013-01-01 01:30:00,7.757606e+06,2.819783e+06,3.258710e+06,2.570549e+06,5.552388e+06,5.987886e+06,1.195982e+07,3.328365e+06,5.844183e+06,5.683886e+06,3.660857e+06,4.953680e+06,6.369792e+07
2013-01-01 02:00:00,7.757609e+06,2.819783e+06,3.258711e+06,2.570550e+06,5.552389e+06,5.987887e+06,1.195982e+07,3.328366e+06,5.844185e+06,5.683889e+06,3.660859e+06,4.953682e+06,6.369794e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 22:00:00,8.197316e+06,2.786297e+06,3.429878e+06,2.572278e+06,5.562262e+06,5.980698e+06,1.235893e+07,3.317024e+06,6.110360e+06,6.100996e+06,3.907420e+06,5.160086e+06,6.583479e+07
2022-12-31 22:30:00,8.197318e+06,2.786297e+06,3.429879e+06,2.572278e+06,5.562262e+06,5.980698e+06,1.235893e+07,3.317023e+06,6.110361e+06,6.100998e+06,3.907422e+06,5.160088e+06,6.583480e+07
2022-12-31 23:00:00,8.197320e+06,2.786297e+06,3.429880e+06,2.572278e+06,5.562262e+06,5.980698e+06,1.235893e+07,3.317023e+06,6.110362e+06,6.101000e+06,3.907423e+06,5.160089e+06,6.583482e+07


In [87]:
filename = f"./data/jours_feries_metropole.csv"
df = pd.read_csv(filename, sep=",")
df = df[["date"]]
#keep rows with dates between 2013 and 2023
df = df[df.date >= "2013-01-01"]
df = df[df.date <= "2023-12-31"]

#create new dataframe with all dates between 2013 and 2023 and if in jours_feries, put 1 else 0
jours_feries = pd.DataFrame({"Date": pd.date_range(start="2013-01-01", end="2023-12-31")})
jours_feries["Jours_feries"] = jours_feries["Date"].isin(df["date"]).astype(int)

print(jours_feries)

                     Jours_feries
Date                             
2013-01-01 00:00:00           1.0
2013-04-01 00:00:00           1.0
2013-05-01 00:00:00           1.0
2013-05-08 00:00:00           1.0
2013-05-08 00:30:00           1.0
...                           ...
2023-07-14 00:00:00           1.0
2023-08-15 00:00:00           1.0
2023-11-01 00:00:00           1.0
2023-11-11 00:00:00           1.0
2023-12-25 00:00:00           1.0

[168 rows x 1 columns]


In [89]:
#create model with conso_national, national_meteo and nb_habitants
model = pd.concat([conso_national, national_meteo, nb_habitants], axis=1)
model

In [ ]:
#transform model in time series
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import pyplot
result = seasonal_decompose(model["Consommation"], model="additive")
result.plot()
pyplot.show()

In [ ]:
#test sarima mdodel
from statsmodels.tsa.statespace.sarimax import SARIMAX
from random import random
# contrived dataset
data = [x + random() for x in range(1, 100)]
# fit model
model = SARIMAX(data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 1))
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.predict(len(data), len(data))
print(yhat)



In [ ]:
#test auto.arima
from pmdarima.arima import auto_arima
# fit model
model = auto_arima(data, start_p=1, start_q=1,
                   max_p=3, max_q=3, m=12,
                   start_P=0, seasonal=True,
                   d=1, D=1, trace=True,
                   error_action='ignore',
                   suppress_warnings=True,
                   stepwise=True)
print(model.summary())


In [ ]:
#train test split of model
train = model[:"2020-12-31"]
test = model["2021-01-01":]

#train test split of conso_national
train_conso = conso_national[:"2020-12-31"]
test_conso = conso_national["2021-01-01":]

In [ ]:
#test prophet
from pandas import read_csv
from pandas import to_datetime
from pandas import DataFrame
from fbprophet import Prophet

# prepare expected column names
def prepare_dataset(df, date_col, target_col):
    df = df[[date_col, target_col]]
    df.columns = ['ds', 'y']
    return df

# load the dataset
df = read_csv('./data/eco2mix/France/En-cours-Consolide.txt', sep='\t', encoding="latin-1")
# prepare expected column names
df = prepare_dataset(df, 'Date', 'Consommation')
# define the model
model = Prophet()
# fit the model
model.fit(df)
# define the period for which we want a prediction
future = list()
for i in range(1, 13):
    date = '2021-%02d' % i
    future.append([date])
future = DataFrame(future)
future.columns = ['ds']
future['ds']= to_datetime(future['ds'])
# use the model to make a forecast
forecast = model.predict(future)
# summarize the forecast
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])
# plot forecast
model.plot(forecast)
pyplot.show()
